In [ ]:
'''Calculate qc statistics and renaming variables to gene symbol
author: Carsten Knutsen
Date: March 18 2023
conda environment: bulk_rnaseq
'''

In [ ]:
import pandas as pd
import os
from anndata import AnnData
import seaborn as sns
import numpy as np
from gtfparse import read_gtf


In [ ]:
##Input and output files
output = 'data/output_data/'
input_counts = 'data/aligned_sequencing/counts.csv'
map_qc = f'{output}/mapping_qc.csv'
output_counts = f'{output}/clean_counts.csv'
##read in original counts
df = pd.read_csv(input_counts,index_col=0)
qc = df.iloc[:4]
counts= df.iloc[4:]
qc = qc.append(pd.Series(counts.sum(axis=0), name='mapped_successful'))
qc = qc.T
##Calculate QC for mapping
total = qc.sum(axis = 1)
perc_df = qc.div(total, axis = 0) *100
perc_df.columns = [f'% {x}' for x in perc_df.columns ]
perc_df['total_reads'] = total
perc_df.sort_index().to_csv(f'{map_qc}')
##Rename ensembl to gene
gtf = read_gtf('/media/carsten/hdd/genomes/mouse/gencode.vM27.annotation.gtf')
gene_name_dict = pd.Series(gtf.gene_name.values, index=gtf.gene_id).to_dict()
counts.index = [gene_name_dict[x] for x in counts.index]
counts = counts.T.sum(axis=1, level=0).T
counts.index.names = ['symbol']
counts[sorted(counts.columns.tolist())].to_csv(output_counts)